In [251]:
import numpy as np
import pandas as pd
import os
import sys
import csv

In [252]:
data_folder_lineup = 'NBA Hackathon - Game Lineup Data Sample (50 Games).txt'
file_lineup = open(data_folder_lineup,'r')
game_id = -1
period = -1
person_id = -1
team_id = -1
status = 'N'
i = 0
lineupt2p = {}
lineupp2t = {}
for line in file_lineup:
    if i != 0:
        data = line.strip().split('\t')    
        lineupt2p[data[0]] = {}
        lineupp2t[data[0]] = {}
    i += 1
i = 0
file_lineup.close()
file_lineup = open(data_folder_lineup,'r')
for line in file_lineup:
    if i != 0:
        data = line.strip().split('\t')    
        lineupt2p[data[0]][data[1]] = {}
    i += 1
i = 0
file_lineup.close()
file_lineup = open(data_folder_lineup,'r')
for line in file_lineup:
    if i != 0:
        data = line.strip().split('\t')    
        lineupt2p[data[0]][data[1]][data[3]] = []
    i += 1
i = 0
file_lineup.close()
file_lineup = open(data_folder_lineup,'r')
for line in file_lineup:
    if i != 0:
        data = line.strip().split('\t')
        if len(data) == 5:
            game_id = data[0]
            period = data[1]
            person_id = data[2]
            team_id = data[3]
            status = data[4]
        else:
            print('error')
            break
        lineupt2p[game_id][period][team_id].append(person_id)
        lineupp2t[game_id][person_id] = team_id
    i += 1
file_lineup.close()
#lineup[game_id][period][team_id] = (person_id)

In [253]:
data_folder_pbp = 'NBA Hackathon - Play by Play Data Sample (50 Games).txt'
file_pbp = open(data_folder_pbp,'r')
game_id = -1
event_num = -1
event_msg_type = -1
period = -1
wc_time = -1
pc_time = -1
action_type = -1
option1 = -1
option2 = -1
option3 = -1
team_id = -1
person1 = -1
person2 = -1
team_id_type = -1
i = 0
pbp = {}
for line in file_pbp:
    if i != 0:
        data = line.strip().split('\t')
        pbp[data[0]] = {}
    i += 1
i = 0
file_pbp.close()
file_pbp = open(data_folder_pbp,'r')
for line in file_pbp:
    if i != 0:
        data = line.strip().split('\t')
        pbp[data[0]][data[3]] = []
    i += 1
i = 0
file_pbp.close()
file_pbp = open(data_folder_pbp,'r')
for line in file_pbp:
    if i != 0:
        data = line.strip().split('\t')
        if len(data) == 14:
            game_id = data[0]
            event_num = data[1]
            event_msg_type = data[2]
            period = data[3]
            wc_time = data[4]
            pc_time = data[5]
            action_type = data[6]
            option1 = data[7]
            option2 = data[8]
            option3 = data[9]
            team_id = data[10]
            person1 = data[11]
            person2 = data[12]
            team_id_type = data[13]
        else:
            print('error')
            break
        pbp[game_id][period].append((event_num,event_msg_type,wc_time,pc_time,
                                     action_type,option1,option2,option3,team_id,person1,person2,team_id_type))
    i += 1
file_pbp.close()
#pbp[game_id][period] = (event_num,event_msg_type,wc_time,pc_time,
#                                    action_type,option1,option2,option3,team_id,person1,person2,team_id_type))

In [254]:
res = {}
for game in pbp:
    example = pbp[game].copy()
    p2t = lineupp2t[game].copy()
    res[game] = p2t.copy()
    total_ac_ply = {}
    for player in res[game]:
        res[game][player] = 0
    for q in example:
        total_ac_ply[q] = {}
        for data in example[q]:
            person2 = data[10]
            team_id = data[8]
            res[game][person2] = 0
    for q in example:
        act_players = lineupt2p[game][q].copy()
        for data in example[q]:
            event_msg_type = data[1]
            pc_time = data[3]
            option1 = data[5]
            team_id = data[8]
            person1 = data[9]
            person2 = data[10]
            if pc_time not in total_ac_ply[q]:
                total_ac_ply[q][pc_time] = act_players
            if event_msg_type == '1':
                score = int(option1)
                for team in act_players:
                    players = act_players[team]
                    if team == team_id:
                        for player in players:
                            res[game][player] += score
                    else:
                        for player in players:
                            res[game][player] -= score
            if event_msg_type == '3' and option1 == '1':
                score = int(option1)
                act_players_foul = total_ac_ply[q][pc_time]
                for team in act_players_foul:
                    players = act_players_foul[team]
                    if team == team_id:
                        for player in players:
                            res[game][player] += score
                    else:
                        for player in players:
                            res[game][player] -= score
            if event_msg_type == '8':
                team = p2t[person1]
                act_players[team].remove(person1)
                act_players[team].append(person2)
                p2t[person2] = team
csv_file = open('double_legs_Q1_BBALL.csv','w',newline='')
writer1 = csv.writer(csv_file)
writer1.writerow(['game_id','player_id','plus/minus'])
for key in res:
    for ply in res[key]:
        writer1.writerow([key,ply,res[key][ply]])
csv_file.close()